In [1]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm

## Phase 2.2 Data Pre Processing
### Data collection and preprocessing

I have downloaded sales data from an MLS system. It only allows to download up to 3,000 records, so I have ended up with a bunch of files. Every export operation has taken quite some time, so what seemed to be an easy task turned out to be a tedious and tiresome process. One would expect to get clean data as a result of this little torture, but it has turned out to be anything but pure.

Luckily Pandas is a perfect tool for cleaning. Starting with 117MB, 112970 entries, and 135 columns, I have squeezed it down to 27.2 Mb, the same amount of listings, and 69 columns without losing any meaningful information.

### Technologies

I plan to use the dataset with Kepler.gl, an extraordinary framework for geospatial time-series visualizations. However, before loading data to Kepler, I still need to do quite a bit of modeling and calculations. First, I need to geotag the locations and build a hex grid using the uber H3 framework. Then I need to train a deep neural network model to get embeddings. Finally, I will have to apply a clusterization algorithm to the hex map to define existing submarkets. Then I will be able to show how price levels evolve in different submarkets. 

A bonus problem would be to show how borders of the submarkets evolve. However, to visualize this, I would have to enrich my data set with historical information from census tracts. This task would not be too hard with the given framework, but extracting the data from IPUMS USA takes time. Given the complexity of my endeavor, I wisely restrict the current scope of work to the given dataset.

In [2]:
# List of the csvs with different # of columns:
diff_list = ['C:\\SFSU\\Spring 2020\\Project\\Data\\sales\\Spreadsheet.csv',
             'C:\\SFSU\\Spring 2020\\Project\\Data\\sales\\Spreadsheet (1).csv',
             'C:\\SFSU\\Spring 2020\\Project\\Data\\sales\\Spreadsheet (2).csv',
             'C:\\SFSU\\Spring 2020\\Project\\Data\\sales\\Spreadsheet (3).csv']

# List of csvs with regular # of columns to import:
path = r'C:\SFSU\Spring 2020\Project\Data\sales' # use your path
all_files = glob.glob(path + "/*.csv")
all_files = list(set(all_files) - set(diff_list))

In [3]:
# List of dataframes to concatenate:
## Add dataframes with regular columns:
li = []
for filename in tqdm(all_files):
    df = pd.read_csv(filename)
    li.append(df)

## Find intersection of columns with the regular columns set:
df2 = pd.read_csv(diff_list[0])
import_cols = (df2.columns).intersection(set(df.columns))
li.append(df2.loc[:,import_cols])
    
for filename in tqdm(diff_list[1:]):
    df = pd.read_csv(filename, usecols=import_cols)
    li.append(df)
    
# Concat the imported dataframes and drop duplicates:
frame = pd.concat(li, axis=0, ignore_index=True).drop_duplicates()
frame.info()

100%|██████████| 3/3 [00:00<00:00, 15.98it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 112970 entries, 0 to 126943
Data columns (total 81 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   MLS #                        112970 non-null  object 
 1   #Pics                        112970 non-null  int64  
 2   Status                       112970 non-null  object 
 3   Address                      112970 non-null  object 
 4   Price                        112970 non-null  object 
 5   DOM                          112970 non-null  object 
 6   Beds                         111492 non-null  float64
 7   Baths(F)                     112918 non-null  float64
 8   Baths(P)                     108577 non-null  float64
 9   SqFt                         112776 non-null  float64
 10  LotSqFt                      112205 non-null  object 
 11  City                         112970 non-null  object 
 12  Class                        112970 non-null  object 
 13 

In [4]:
# Basic cleaning.
# Convert object columns (such as price per sq.ft. and price) to numeric. 
obj_columns = list(frame.columns[frame.dtypes=='object'])
for col in tqdm(obj_columns):
    frame[col] = pd.to_numeric(frame[col].replace('[\$\,]', '', regex=True), errors='ignore')
    
    # Convert date-time columns:
    if frame[col].dtype != 'int64' and frame[col].dtype != 'float64':
        frame[col] = pd.to_datetime(frame[col], errors='ignore')
        
# Drop mostly empty columns
# (We can return later to see if there is any signal in the dropped columns.):
check_columns = list(frame.columns[frame.count()>=5000])
drop_columns = list(frame.columns[frame.count()<5000])

# Drop columns that have only zeroes:
max_count = frame[check_columns].describe().T['max']
drop_columns += list(max_count[max_count==0].index)
frame.drop(axis=1, columns=drop_columns, inplace=True)

frame.info()

100%|██████████| 56/56 [00:11<00:00,  4.89it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 112970 entries, 0 to 126943
Data columns (total 66 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   MLS #                        112970 non-null  object        
 1   #Pics                        112970 non-null  int64         
 2   Status                       112970 non-null  object        
 3   Address                      112970 non-null  object        
 4   Price                        112970 non-null  int64         
 5   DOM                          112970 non-null  object        
 6   Beds                         111492 non-null  float64       
 7   Baths(F)                     112918 non-null  float64       
 8   Baths(P)                     108577 non-null  float64       
 9   SqFt                         112776 non-null  float64       
 10  LotSqFt                      112205 non-null  float64       
 11  City                      

In [5]:
# Check date time columns:
frame.dtypes[frame.dtypes=='<M8[ns]']

Listing Date          datetime64[ns]
Original List Date    datetime64[ns]
Status Change Date    datetime64[ns]
dtype: object

In [6]:
# Try to find categorical variabeles:
obj_count = frame[frame.dtypes[frame.dtypes=='object'].index].nunique()
cat_col = obj_count[obj_count <= 373]
cat_col.sort_values()

How Sold                         1
City                             1
Class                            1
Lot Size Area Maximum Units      2
Fireplace YN                     2
HOA Exist Y/N                    2
Status                           2
Internet Listing YesNo           2
HOA Fee Frequency                3
Geocode Quality                  4
PropType                         5
Occupied By                      5
Source MLS                       7
Structure SqFt Source           13
Area                           115
Zip Code                       310
Zoning Text                    347
dtype: int64

In [7]:
# Zip Code column has too many values:
frame['Zip Code'].astype(str).map(len).max()

10

In [8]:
# Find 5 digits zip codes:
zip5=frame['Zip Code'].astype(str).str.extract(r'(\d{5})')[0]

# Extract 4 digit codes from non 5 digits zip codes:
m = zip5.isna()
frame.loc[m,'Zip Code'] = frame['Zip Code'].loc[m].astype(str).str.extract(r'(\d{4})')[0]
frame.loc[-m,'Zip Code'] = zip5

In [9]:
# Drop categorical variables with only one category:
drop_columns = cat_col[cat_col==1].index
frame.drop(axis=1, columns=drop_columns, inplace=True)

# Convert remaining variables to categorical:
obj_count = frame[frame.dtypes[frame.dtypes=='object'].index].nunique()
cat_col = obj_count[obj_count <= 373].index
for col in tqdm(cat_col):
    frame[col] = frame[col].astype('category')

100%|██████████| 14/14 [00:00<00:00, 30.58it/s]


### Memory reducer:
Thanks You Guillaume Martin for the Awesome Memory Optimizer! This is considered a standard memory reducer by now. And is widely used throughout differenet competitions.

https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

In [10]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else: df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [11]:
# Reduce memory:
num_col = frame.select_dtypes(include=['int16','int64','float64']).columns
frame.loc[:,num_col] = reduce_mem_usage(frame.loc[:,num_col])

# Print dataframe info sorted by value count:
sorted_cols = frame.count().sort_values(ascending=False).index
frame[sorted_cols].info()

Memory usage of dataframe is 37.75 MB
Memory usage after optimization is: 16.85 MB
Decreased by 55.4%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 112970 entries, 0 to 126943
Data columns (total 63 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   MLS #                        112970 non-null  object        
 1   Address                      112970 non-null  object        
 2   Area                         112970 non-null  category      
 3   Days On Market               112970 non-null  int16         
 4   Geocode Quality              112970 non-null  category      
 5   Internet Listing YesNo       112970 non-null  category      
 6   List Price                   112970 non-null  int32         
 7   Listing Date                 112970 non-null  datetime64[ns]
 8   DOM                          112970 non-null  object        
 9   Price                        112970 non-null  int32     

In [12]:
# Export clean dataframe:
filename = path + 'frame.csv'
frame.to_csv(filename)